In [1]:
cd /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DCTdiff_Github

/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DCTdiff_Github


## Package Installations

In [ ]:
# Required Softwares for training DCTDiff 
!python3 -m venv dctdiff-env
!source dctdiff-env/bin/activate

!pip install matplotlib
!pip install accelerate==0.33.0   # (auto install pytorch 2.4)
!pip install absl-py ml_collections einops wandb ftfy==6.1.1 transformers==4.23.1
!pip install opencv-python
!pip install scipy

# xformers is optional, but it would greatly speed up the attention computation.
!pip install -U xformers
!pip install torchvision==0.19.0
!pip install pytorch-fid
!pip install pandas


In [ ]:
# Additional Packages installed due to run time issues
# Uninstall conflicting packages
!pip uninstall -y numpy opencv-python-headless scipy

# Install numpy compatible with your environment
!pip install numpy==1.26.4

# Install opencv-python-headless version compatible with numpy 1.x
!pip install opencv-python-headless==4.7.0.72

# Reinstall scipy (latest compatible)
!pip install scipy==1.11.1


## Train and Test Data split for DCTDiff

In [ ]:
# This Model needs Train and Test set in subfolders - Creating Dataset for training
import os
import shutil

# --- CONFIGURATION ---
source_dir = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64'   # Source folder
output_dir_1 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/DCTdiff_Train/64_Train'  # 80% Train
output_dir_2 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/DCTdiff_Test/64_Test'   # 20% Test

# --- CREATE OUTPUT DIRECTORIES IF THEY DON'T EXIST ---
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)

# --- GET SORTED FILE LIST ---
all_files = sorted([
    f for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
])

# --- CALCULATE SPLIT INDEX ---
split_index = int(0.8 * len(all_files))

# --- SPLIT FILES SEQUENTIALLY ---
files_A = all_files[:split_index]   # 80%
files_B = all_files[split_index:]   # 20%

# --- COPY FILES ---
for f in files_A:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_1, f))

for f in files_B:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_2, f))

print(f"✅ Done. {len(files_A)} files → {output_dir_1}, {len(files_B)} files → {output_dir_2}")


In [1]:
# checking image count for each category Member & Non-Member of both celeba64 dataset created for DCTDiff
import os
print("Total images 64:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64")))
print("Total images 64_Train:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/DCTdiff_Train/64_Train")))
print("Total images 64_Test:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/DCTdiff_Test/64_Test")))

Total images 64: 202599
Total images 64_Train: 162079
Total images 64_Test: 40520


## DCTDiff Model Training

In [ ]:
# DCTDiff Train Code - Original code was running with 4 GPU, due to limitation I modified it as 1
!module load X11  # if needed for libX11.so.6

!accelerate launch --num_processes 1 --mixed_precision fp16 train.py --config=configs/celeba64_uvit_small_2by2.py --workdir checkpoints/celeba64


## Evaluation for DCT model using 3 samplers DPM-Solver, Euler ODE and Euler SDE
Created trainref code only targeting evaluation and sample Generation, incorporated from train.py (original source codes, Nothing ivented)
scores are available under checkpoint-> eval_dpm_solver_10000_check, eval_euler_maruyama_ode_10000_check, eval_euler_maruyama_sde_10000_check

In [ ]:
# Need to create FID Stats for our training data
!python -m pytorch_fid --save-stats /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train assets/fid_stats/64_Train_stats

In [1]:
# Evaluation scores for checkpoint 400K step
from trainref import eval_step
import torch
import os
import ml_collections
import utils
import sde
import accelerate
from datasets import get_dataset
from DCT_utils import zigzag_order, reverse_zigzag_order
from utils import initialize_train_state

# --- Load Config ---
from configs.celeba64_uvit_small_2by2 import get_config

config = get_config()
config.workdir = "checkpoints/celeba64"
config.ckpt_root = os.path.join(config.workdir, 'ckpts')
config.sample_dir = os.path.join(config.workdir, 'samples')
config = ml_collections.FrozenConfigDict(config)

# --- Setup Accelerator and Device ---
accelerator = accelerate.Accelerator()
device = accelerator.device

# --- Load Dataset ---
dataset = get_dataset(**config.dataset)
low2high_order = zigzag_order(config.dataset.block_sz)
reverse_order = reverse_zigzag_order(config.dataset.block_sz)

# --- Load Model from Checkpoint ---
train_state = initialize_train_state(config, device)
train_state.resume(config.ckpt_root)
print("Step:", train_state.step)

nnet_ema = train_state.nnet_ema
score_model_ema = sde.ScoreModel(
    nnet=nnet_ema,
    pred=config.pred,
    sde=sde.VPSDE(SNR_scale=config.dataset.SNR_scale)
)

for algo in ["euler_maruyama_ode", "euler_maruyama_sde", "dpm_solver"]:
    for steps in [10, 20, 50, 100]:
        out_path = f"{config.sample.path}_{algo}_nfe{steps}_snr4"
        print(f"\n🌀 Evaluating {algo.upper()} | NFE={steps} | SNR=4.0")

        fid = eval_step(
            n_samples=config.sample.n_samples,
            sample_steps=steps,
            algorithm=algo,
            Y_bound=config.dataset.Y_bound,
            path=out_path,
            config=config,
            dataset=dataset,
            train_state=train_state,
            score_model_ema=score_model_ema,
            reverse_order=reverse_order,
            accelerator=accelerator,
            device=device
        )
        print(f"✅ {algo.upper()} (NFE={steps}, SNR=4.0) → FID: {fid:.4f}")


/rds/homes/k/kxr414/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


using eta [244.925] for training
attention mode is flash


/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DCTdiff_Github/utils.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.step = torch.load(os.path.join(path, 'step.pth'

Step: 400000
using SNR_scale: 4.0 for training
ScoreModel with pred=noise_pred, sde=vpsde beta_0=0.1 beta_1=20, T=1

🌀 Evaluating EULER_MARUYAMA_ODE | NFE=10 | SNR=4.0
using eta (244.925,) for sampling


sample2dir:   0%|          | 0/21 [00:00<?, ?it/s]

...

using SNR scale: 4.0 for DPM-Solver sampling


sample2dir: 100%|██████████| 21/21 [28:11<00:00, 80.54s/it]


generated 10000 images...


100%|██████████| 200/200 [00:06<00:00, 30.79it/s]


✅ DPM_SOLVER (NFE=100, SNR=4.0) → FID: 5.8923


In [2]:
# Evaluation scores for checkpoint 250K(for cross verifying how these 2 checkpoints are trained with FID scores
from trainref import eval_step
import torch
import os
import ml_collections
import utils
import sde
import accelerate
from datasets import get_dataset
from DCT_utils import zigzag_order, reverse_zigzag_order
from utils import initialize_train_state

# --- Load Config ---
from configs.celeba64_uvit_small_2by2 import get_config

config = get_config()
config.workdir = "checkpoints/celeba64"
config.ckpt_root = os.path.join(config.workdir, 'ckpts_250000')
config.sample_dir = os.path.join(config.workdir, 'samples')
config = ml_collections.FrozenConfigDict(config)

# --- Setup Accelerator and Device ---
accelerator = accelerate.Accelerator()
device = accelerator.device

# --- Load Dataset ---
dataset = get_dataset(**config.dataset)
low2high_order = zigzag_order(config.dataset.block_sz)
reverse_order = reverse_zigzag_order(config.dataset.block_sz)

# --- Load Model from Checkpoint ---
train_state = initialize_train_state(config, device)
train_state.resume(config.ckpt_root)
print("Step:", train_state.step)

nnet_ema = train_state.nnet_ema
score_model_ema = sde.ScoreModel(
    nnet=nnet_ema,
    pred=config.pred,
    sde=sde.VPSDE(SNR_scale=config.dataset.SNR_scale)
)

for algo in ["euler_maruyama_ode", "euler_maruyama_sde", "dpm_solver"]:
    for steps in [10, 20, 50, 100]:
        out_path = f"{config.sample.path}_{algo}_nfe{steps}_snr4"
        print(f"\n🌀 Evaluating {algo.upper()} | NFE={steps} | SNR=4.0")

        fid = eval_step(
            n_samples=config.sample.n_samples,
            sample_steps=steps,
            algorithm=algo,
            Y_bound=config.dataset.Y_bound,
            path=out_path,
            config=config,
            dataset=dataset,
            train_state=train_state,
            score_model_ema=score_model_ema,
            reverse_order=reverse_order,
            accelerator=accelerator,
            device=device
        )
        print(f"✅ {algo.upper()} (NFE={steps}, SNR=4.0) → FID: {fid:.4f}")


using eta [244.925] for training
Step: 250000
using SNR_scale: 4.0 for training
ScoreModel with pred=noise_pred, sde=vpsde beta_0=0.1 beta_1=20, T=1

🌀 Evaluating EULER_MARUYAMA_ODE | NFE=10 | SNR=4.0
using eta (244.925,) for sampling


sample2dir:   0%|          | 0/21 [00:00<?, ?it/s]

euler_maruyama with sample_steps=10


sample2dir:   5%|▍         | 1/21 [00:11<03:52, 11.62s/it]

euler_maruyama with sample_steps=10


...

using SNR scale: 4.0 for DPM-Solver sampling


sample2dir: 100%|██████████| 21/21 [28:09<00:00, 80.45s/it]


generated 10000 images...


100%|██████████| 200/200 [00:06<00:00, 30.84it/s]


✅ DPM_SOLVER (NFE=100, SNR=4.0) → FID: 5.5913


## Generating Samples using trained DCTDiff Model for 3 samplers with 100 NFE's

In [1]:
from trainref import eval_step
import torch
import os
import ml_collections
import utils
import sde
import accelerate
from datasets import get_dataset
from DCT_utils import zigzag_order, reverse_zigzag_order
from utils import initialize_train_state

# --- Load Config ---
from configs.celeba64_uvit_small_2by2 import get_config

config = get_config()
config.workdir = "checkpoints/celeba64"
config.ckpt_root = os.path.join(config.workdir, 'ckpts')
config.sample_dir = os.path.join(config.workdir, 'samples')
config = ml_collections.FrozenConfigDict(config)

# --- Setup Accelerator and Device ---
accelerator = accelerate.Accelerator()
device = accelerator.device

# --- Load Dataset ---
dataset = get_dataset(**config.dataset)
low2high_order = zigzag_order(config.dataset.block_sz)
reverse_order = reverse_zigzag_order(config.dataset.block_sz)

# --- Load Model from Checkpoint ---
train_state = initialize_train_state(config, device)
train_state.resume(config.ckpt_root)
print("Step:", train_state.step)

nnet_ema = train_state.nnet_ema
score_model_ema = sde.ScoreModel(
    nnet=nnet_ema,
    pred=config.pred,
    sde=sde.VPSDE(SNR_scale=config.dataset.SNR_scale)
)

for algo in ["euler_maruyama_ode", "euler_maruyama_sde", "dpm_solver"]:
    for steps in [100]:
        out_path = f"{config.sample.path}_{algo}_nfe{steps}_png"
        print(f"\n🌀 Evaluating {algo.upper()} | NFE={steps} | SNR=4.0")

        fid = eval_step(
            n_samples=config.sample.n_samples,
            sample_steps=steps,
            algorithm=algo,
            Y_bound=config.dataset.Y_bound,
            path=out_path,
            config=config,
            dataset=dataset,
            train_state=train_state,
            score_model_ema=score_model_ema,
            reverse_order=reverse_order,
            accelerator=accelerator,
            device=device
        )
        print(f"✅ {algo.upper()} (NFE={steps}, SNR=4.0) → FID: {fid:.4f}")


/rds/homes/k/kxr414/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


using eta [244.925] for training
attention mode is flash


/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DCTdiff_Github/utils.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.step = torch.load(os.path.join(path, 'step.pth'

Step: 400000
using SNR_scale: 4.0 for training
ScoreModel with pred=noise_pred, sde=vpsde beta_0=0.1 beta_1=20, T=1

🌀 Evaluating EULER_MARUYAMA_ODE | NFE=100 | SNR=4.0
using eta (244.925,) for sampling


sample2dir:   0%|          | 0/21 [00:00<?, ?it/s]

...

using SNR scale: 4.0 for DPM-Solver sampling


sample2dir: 100%|██████████| 21/21 [31:03<00:00, 88.74s/it]


generated 10000 images...


100%|██████████| 200/200 [00:06<00:00, 28.58it/s]


Samples_dpm_solver_nfe100.npz
✅ DPM_SOLVER (NFE=100, SNR=4.0) → FID: 5.3251
